In [1]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
df=pd.read_csv("/kaggle/input/sri-lanka-job-market-dataset-2025/Job Market Sri Lanka.csv")
df.head()

,year,month,job_title,industry,experience_level,salary_min_gbp,salary_max_gbp,salary_median_gbp,number_of_openings,demand_index,...,job_type,skills_required,job_title_encoded,industry_encoded,city_encoded,region_encoded,education_requirement_encoded,job_type_encoded,skills_required_encoded,hiring_trend_encoded
0,2025,2,Pharmacist,Retail,Senior,4819,7033,6224,8,58,...,Temporary,Customer Service,7,9,10,8,4,4,366,2
1,2025,5,Teacher,Telecoms,Senior,8223,14740,12703,6,38,...,Internship,"Accounting, SQL",11,10,10,6,1,2,111,2
2,2025,8,Sales Executive,Telecoms,Senior,10394,20832,13701,2,41,...,Part-time,"Accounting, Python, Project Management",9,10,9,7,2,3,109,2
3,2025,9,Sales Executive,Logistics,Entry,1795,4620,2814,5,26,...,Part-time,"Clinical Care, Communication, Accounting",9,7,11,2,0,3,135,0
4,2025,8,Sales Executive,Finance,Senior,14273,20999,18720,3,56,...,Temporary,"Data Analysis, Clinical Care, Excel",9,3,7,0,1,4,504,2


In [3]:
df.shape

(30000, 24)

In [4]:
df.isnull().sum()

year                             0
month                            0
job_title                        0
industry                         0
experience_level                 0
salary_min_gbp                   0
salary_max_gbp                   0
salary_median_gbp                0
number_of_openings               0
demand_index                     0
hiring_trend                     0
city                             0
region                           0
education_requirement            0
job_type                         0
skills_required                  0
job_title_encoded                0
industry_encoded                 0
city_encoded                     0
region_encoded                   0
education_requirement_encoded    0
job_type_encoded                 0
skills_required_encoded          0
hiring_trend_encoded             0
dtype: int64

In [5]:
df.dtypes

year                              int64
month                             int64
job_title                        object
industry                         object
experience_level                 object
salary_min_gbp                    int64
salary_max_gbp                    int64
salary_median_gbp                 int64
number_of_openings                int64
demand_index                      int64
hiring_trend                     object
city                             object
region                           object
education_requirement            object
job_type                         object
skills_required                  object
job_title_encoded                 int64
industry_encoded                  int64
city_encoded                      int64
region_encoded                    int64
education_requirement_encoded     int64
job_type_encoded                  int64
skills_required_encoded           int64
hiring_trend_encoded              int64
dtype: object

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   year                           30000 non-null  int64 
 1   month                          30000 non-null  int64 
 2   job_title                      30000 non-null  object
 3   industry                       30000 non-null  object
 4   experience_level               30000 non-null  object
 5   salary_min_gbp                 30000 non-null  int64 
 6   salary_max_gbp                 30000 non-null  int64 
 7   salary_median_gbp              30000 non-null  int64 
 8   number_of_openings             30000 non-null  int64 
 9   demand_index                   30000 non-null  int64 
 10  hiring_trend                   30000 non-null  object
 11  city                           30000 non-null  object
 12  region                         30000 non-null  object
 13  e

In [7]:
df.head(2)

,year,month,job_title,industry,experience_level,salary_min_gbp,salary_max_gbp,salary_median_gbp,number_of_openings,demand_index,...,job_type,skills_required,job_title_encoded,industry_encoded,city_encoded,region_encoded,education_requirement_encoded,job_type_encoded,skills_required_encoded,hiring_trend_encoded
0,2025,2,Pharmacist,Retail,Senior,4819,7033,6224,8,58,...,Temporary,Customer Service,7,9,10,8,4,4,366,2
1,2025,5,Teacher,Telecoms,Senior,8223,14740,12703,6,38,...,Internship,"Accounting, SQL",11,10,10,6,1,2,111,2


In [8]:

df.corr(numeric_only=True)['salary_median_gbp']


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


year                                  NaN
month                            0.011911
salary_min_gbp                   0.983400
salary_max_gbp                   0.956296
salary_median_gbp                1.000000
number_of_openings              -0.003132
demand_index                     0.000006
job_title_encoded                0.033565
industry_encoded                 0.041765
city_encoded                     0.000213
region_encoded                   0.017124
education_requirement_encoded    0.006281
job_type_encoded                -0.006876
skills_required_encoded          0.007301
hiring_trend_encoded            -0.001308
Name: salary_median_gbp, dtype: float64

In [9]:
select=['job_title','experience_level','industry','city','salary_median_gbp']
data=df[select]
data=data.dropna()
data=data.drop_duplicates()
data.dtypes,data.shape

(job_title            object
 experience_level     object
 industry             object
 city                 object
 salary_median_gbp     int64
 dtype: object,
 (27822, 5))

In [10]:
df['job_title'] = df['job_title'].str.strip()
df['industry'] = df['industry'].str.strip()
df['city'] = df['city'].str.strip()
data.dtypes

job_title            object
experience_level     object
industry             object
city                 object
salary_median_gbp     int64
dtype: object

preprocessing

In [11]:
data['job_title'].value_counts()

job_title
Software Engineer    2512
Marketing Manager    2463
Project Manager      2446
Data Scientist       2436
Pharmacist           2425
Civil Engineer       2386
Accountant           2379
Sales Executive      2349
HR Specialist        2327
Teacher              2173
Nurse                2105
Customer Support     1821
Name: count, dtype: int64

In [12]:
data['experience_level'].value_counts()

experience_level
Senior    10118
Mid        9412
Entry      8292
Name: count, dtype: int64

In [13]:
data['industry'].value_counts()

industry
Healthcare       2584
IT               2572
Telecoms         2534
Education        2466
Finance          2430
Construction     2417
Government       2359
Manufacturing    2314
Logistics        2278
Tourism          2254
Retail           2046
Agriculture      1568
Name: count, dtype: int64

In [14]:
data['city'].value_counts()

city
Batticaloa      2361
Colombo         2354
Jaffna          2351
Matara          2350
Kandy           2344
Trincomalee     2338
Anuradhapura    2337
Kurunegala      2325
Galle           2293
Ratnapura       2271
Negombo         2254
Polonnaruwa     2244
Name: count, dtype: int64

In [15]:
le_job = LabelEncoder()
le_exp = LabelEncoder()
le_ind = LabelEncoder()
le_city = LabelEncoder()

In [16]:
data['job_title'] = le_job.fit_transform(data['job_title'])
data['experience_level'] = le_exp.fit_transform(data['experience_level'])
data['industry'] = le_ind.fit_transform(data['industry'])
data['city'] = le_city.fit_transform(data['city'])

In [17]:
data.head()

,job_title,experience_level,industry,city,salary_median_gbp
0,7,2,9,10,6224
1,11,2,10,10,12703
2,9,2,10,9,13701
3,9,0,7,11,2814
4,9,2,3,7,18720


train model

In [18]:
X = data.drop("salary_median_gbp", axis=1)
y = data["salary_median_gbp"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape,X_test.shape

((22257, 4), (5565, 4))

In [19]:
def model_acc(model):
    model.fit(X_train,y_train)
    acc= model.score(X_test,y_test)
    print(str(model)+ '-->' +str(acc))

In [20]:
lr=LinearRegression()
model_acc(lr)

LinearRegression()-->0.22476025693614943


In [21]:
lasso=Lasso()
model_acc(lasso)

Lasso()-->0.22476542727851934


In [22]:
dt=DecisionTreeRegressor()
model_acc(dt)

DecisionTreeRegressor()-->0.9345832761117714


In [23]:
rf=RandomForestRegressor()
model_acc(rf)

RandomForestRegressor()-->0.9367224955240416


In [24]:
parameters= {'n_estimators':[10,50,100],
             'criterion':['squared_error','absolute_error','poisson']}

grid_obj=GridSearchCV(estimator=rf,param_grid=parameters)
grid_fit=grid_obj.fit(X_train,y_train)

best_model=grid_fit.best_estimator_
best_model

RandomForestRegressor(criterion='absolute_error')

In [25]:
best_model.score(X_test,y_test)

0.9375799014347383

In [26]:
with open('salary_model.pickle', 'wb') as file:
    pickle.dump(best_model, file)